# Рекомендация тарифов

### Цель проекта: построить модель для предложения пользователям нового тарифа с максимальным значением accuracy

### Описание проекта:

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Необходимо построить модель для задачи классификации, которая выберет подходящий тариф.

In [14]:
# Импортируем библиотеки

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

## Загрузка и изучение данных

In [15]:
data = pd.read_csv('/datasets/users_behavior.csv')
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Данные в столбцах 'messages' и 'calls' можно привести к целочисленному типу. 

In [16]:
data['messages'] = data['messages'].astype(int)
data['calls'] = data['calls'].astype(int)

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


In [18]:
# Проверим данные на наличие пропусков

data.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [19]:
# Проверим данные на наличие явных дубликатов

data.duplicated().sum()

0

***Вывод:***

В результате изучения набора данных был изменен тип данных в столбцах messages и calls. Датафрейм состоит из 3214 строк и 5 столбцов. Дубликаты и пропуски не обнаружены.   

## Разделение данных на выборки

In [20]:
# Извлекаем признаки
features = data.drop(['is_ultra'], axis=1)

# Извлекаем целевой признак
target = data['is_ultra']

Спрятанной тестовой выборки нет, поэтому данные нужно разбить на три части: обучающую, валидационную и тестовую. Учитывая, что размеры тестового и валидационного наборов обычно равны, разделим исходные данные в соотношении 3:1:1. Для этого сначала отделим от всего набора данных две равные части, затем разделим их пополам на тестовую и валидационную выборки. 

In [21]:
# Разбиваем данные на обучающую и тестовую выборки в отношении 3:2

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.4, random_state=12345)

In [22]:
# Разбимаем тестовую выборку на валидационную и тестовую в отношении 1:1

features_valid, features_test, target_valid, target_test = train_test_split(
    features_test, target_test, test_size = 0.5, random_state=12345)

In [23]:
print('Размер обучающей выборки:', len(features_train), 'строк')
print('Размер валидационной выборки:', len(features_valid), 'строки')
print('Размер тестовой выборки:', len(features_test), 'строки')

Размер обучающей выборки: 1928 строк
Размер валидационной выборки: 643 строки
Размер тестовой выборки: 643 строки


## Исследование моделей

Целевой признак в нашем случае категориальный, поэтому мы будем решать задачу классификации. Для этого проверим качество моделей DesicionTree, RandomForest и LogisticRegression. Для моделей решающего дерева и случайного леса будем перебирать гиперпараметры в цикле, чтобы найти среди них наиболее точную.

In [24]:
# Decision Tree
# Создаем цикл для глубины решающего дерева от 1 до 10 и найдем модель с максимальной точностью

best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 11):
     model = DecisionTreeClassifier(random_state = 12345, max_depth = depth)
     model.fit(features_train, target_train) 
     predictions_valid = model.predict(features_valid) 
     result = accuracy_score(predictions_valid, target_valid)
     if best_result < result:
             best_model = model
             best_result = result
             best_depth = depth
        
print('Глубина дерева модели с максимальной точностью равна', best_depth)
print('Accuracy модели:', best_result)

Глубина дерева модели с максимальной точностью равна 3
Accuracy модели: 0.7853810264385692


В результате мы получили модель с точностью правильных ответов равной 78.5%

In [25]:
# Random Forest
# Создаем цикл для количества деревьев от 10 до 50, с максимальной глубиной от 1 до 10 и ищем наилучшую модель. 

best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        result = accuracy_score(predictions_valid, target_valid)
        if best_result < result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth
            
print("Accuracy наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", best_depth)



Accuracy наилучшей модели на валидационной выборке: 0.8087091757387247 Количество деревьев: 40 Максимальная глубина: 8


В результате мы получили модель с точностью правильных ответов равной 80%

In [26]:
# Logistic Regression

model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)

result = accuracy_score(predictions_valid, target_valid) 
print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7107309486780715


В результате мы получили модель с точностью правильных ответов равной 71%

***Итоги исследования:***


- Наилучшей является модель случайного леса из 40 деревьев и глубиной равной 8 - ее точность правильных ответов на валидационной выборке равна 80%
- Модель логистической регрессии является наименее точной: ее accuracy равна 71%

## Проверка модели на тестовой выборке

In [18]:
# Создаем модель случайного леса с теми параметрами, которые показали наилучший результат и проверяем ее точность на тестовой выборке

model = RandomForestClassifier(random_state=12345, n_estimators = 40, max_depth = 8)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result_test = accuracy_score(predictions_test, target_test)

print('Точность правильных ответов:', result_test)

Точность правильных ответов: 0.7962674961119751


***Вывод:***

Модель прошла проверку и показала высокий результат на тестовой выборке. Её accuracy равна 79.6%

## Проверка модели на адекватность

Для того, чтобы проверить модель машинного обучения на адекватность, нужно сравнить результаты её тестирования с результатами более простой модели, которая не учитывает никаких особенностей данных и использует случайные значения для классификации. В качестве модели для сравнения будем использовать DummyClassifier из библиотеки scikit-learn. Тест на адекватность не будет пройден, если результаты DummyClassifier окажутся близки к результатам нашей модели.

In [28]:
# Обучаем DummyClassifier на обучающей выборке

model = DummyClassifier(random_state=12345)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result_test = accuracy_score(predictions_test, target_test)

print('Результат модели DummyClassifier:', result_test)

Результат модели DummyClassifier: 0.6842923794712286


***Вывод:***

Точность нашей модели больше, чем точность модели DummyClassifier (79% > 68%). Это означает, что наша модель прошла проверку на адекватность. 

## Итоговый вывод

- В ходе работы был изучен набор данных, содержащий 3214 строки и 5 столбцов. Данные в двух столбцах были приведены к целочисленному типу. В данных не было обнаружено ни пропусков, ни дубликатов. 
- Данные были разбиты на три выборки в отношении 3:1:1 - на обучающую, валидационную и тестовую соответственно. Размер обучающей выборки составляет 1928 строк, а валидационной и тестовой - по 643 строки каждая.
- В ходе исследования проверялось качество моделей трех типов: решающее дерево, случайный лес и логистическая регрессия. Для моделей решающего дерева и случайного леса гиперпараметры перебирались в цикле, чтобы найти среди них наиболее точную. 
- Модель логистической регрессии показала точность, равную 71%. Решающее дерево глубиной 3 показала точность 78.5%. Наилучшей стала модель случайного леса из 40 деревьев и глубиной равной 8: ее точность правильных ответов на валидационной выборке равна 80%.
- Модель с наивысшим показателем точности проверена на тестовой выборке. Её результат accuracy равен 79.6%.
- Дополнительно модель RandomForestClassifier была проверена на адекватность. Для этого результаты её проверки были сравнены с результатами модели DummyClassifier из библиотеки scikit-learn. Модель прошла проверку, потому что её точность оказалась выше.

Таким образом, мы получили модель машинного обучения, которая позволяет решить задачу классификации и с большой точностью подобрать подходящий тариф для клиентов "Мегалайн".